# Text Generation With Markov Chains
The main idea of today's workshop is to write some code that can generate text based on some collection of text (we call this text input a **corpus**). The corpus could be wikipedia articles, the words from novels or a library of tweets, basically any sort of textual data. 

The goal of the process is to gernerate **original** text that is in the same style as the original inputs.

Therea are a number of different techniques that could be used to generate text and for this workshop we have chosen to use markov chains.  

Before we get started we'll need to understand a little about markov chains.

## Markov Chains
Markov chains are used to model stochastic **(random)** processes that have fixed states and transitions between these states that have probabilites attached to them. There's a lot going on in that sentence, so lets have an example! 

I have two choices for how to spend my afternoons. I could go to the gym or play video games. If I go to the gym on one day, I have 50% probability of going to the gym the following day (to try and keep up the good habits). If I play computer games on any afternoon I am highly-likely (80% probability) to play computer games the following day (yes I *am* addicted). 

The transitions from each choice must sum to **1**, so we can infer the chance of me playing video games after going to the gym is 50% and the chance of me going to the gym after playing computer games is 20%. 

The diagram below shows the relationship between the states and transitions. 

![Two state Markov chain](./images/markov_chain.png)


To use the model to generate a sequence of events, we need to pick a start state, and then generate a random value that will decide which state we move to. 

We have two choices when in each state, i.e. remain in the current state or move to the other. We will say that the interval 0 to X represents the chance of staying in the current state so for gym a number generated less than 0.5 will mean staying in the gym state and an number greater than or equal to 0.5 will transision us to the computer games state. For the computer games state Anything less than 0.8 will keep us playing games! 

Lets see this example in action. Highlight the cell below and run it to see what is generated. 

In [7]:
import random #import the random library

DAYS_TO_GENERATE = 5

GYM_GYM = 0.5 #Probabilty of being in gym state and staying there
GYM_GAMES = 0.5 #Probabilty of being in gym state moving to the games state

GAMES_GAMES = 0.8 #We stay playing games if we played today
GAMES_GYM = 0.2 #We go to the gym if we played today.

current_state = "Gym" #Lets start with good intentions! 

print("Starting state {}".format(current_state))

for i in range(DAYS_TO_GENERATE):
    
    random_number = random.random() #Generate a random number
    
    if current_state == "Gym": 
        if random_number < GYM_GYM: #use the gym probabilities 
            current_state = "Gym"
        else:
            current_state = "Games"
    else: 
        if random_number < GAMES_GAMES: #use the games probabilities 
            current_state = "Games"
        else:
            current_state = "Gym"

    print("Day: {} \t Chosen number: {} \t New state: {}".format(i+1, random_number, current_state))

Starting state Gym
Day: 1 	 Chosen number: 0.6230590578557484 	 New state: Games
Day: 2 	 Chosen number: 0.6844879343525906 	 New state: Games
Day: 3 	 Chosen number: 0.3796033546412224 	 New state: Games
Day: 4 	 Chosen number: 0.7399287417471544 	 New state: Games
Day: 5 	 Chosen number: 0.07565741921565827 	 New state: Games


## Activity 1 (5 minutes)
Experiment with the code above. Here are some suggestions: 
1. Change the number of days generated
2. Change the starting state
3. Modify the transition probabilites 

## Data representation 
In the example above we defined variables to store the probabilities of the transitions. With large numbers of states this clearly wont be possible (the maximum number of transitions for $n$ states is $n^{2}$), so even with just 10 states there could be up to 100 transitions. 

Though not the optimal representation, one way to visualise the state transistions would be to place them in an array (or list in python), having them occur the number of times that represent their particular probabilities. 

For our inital example we could have some code as below: 

`
gym_transitions = ["Gym","Games"] #Represents one half probability of each transision
games_transitions = ["Games", "Games, "Games", "Games", "Gym"] #Represents 4/5 or 80% probability of games
`

If we do this then we can make use of the `choice` method in python's library to select from the options randomly.

`current_state = random.choice(games_transitions)`

Lets take a look at the example using the array based scheme. 

**Add in somthing on the hashmpa here**

In [ ]:
gym_transitions = ["Gym","Games"] #Represents one half probability of each transision
games_transitions = ["Games", "Games", "Games", "Games", "Gym"] #Represents 4/5 or 80% probability of games

current_state = "Gym"

for i in range(10):
    if current_state == "Gym":
        current_state = random.choice(gym_transitions)
    else:
        current_state = random.choice(games_transitions)
        
    print("Day: {} \t {}".format(i+1, current_state))    

## Activity 2
There are two things to think about here: 
1. What does changing the representation do for us? 
2. There is a more memory efficent representation of data. Can you figure out what it might be? 

## Getting Started with Text Generation
Now we have a basic understanding of Markov chains, lets take a look at how they might apply in a text scenario. 
We'll use the following statement as a tiny corpus:

"Democracy is the best system of government. Dictatorship is the worst system of government" 

Our first Markov chain considered only a single state when choosing what to do next. If we did that for text generation, we'd generate the next word based only on its predecessor. This would produce text that looks very close to the inputs. If we want to generate original text we'll have to consider more of the previous text to come up with something new. 

We also need to consider what will actually constitute a sentence. For our example we will: 
1. Expect a sentence to start with a capital letter
2. End a sentence with a full stop

The first thing to do is break down our example into two word segments, that we will use to build the keys of our transition matrix.

0. Democracy is
1. is the
2. the best
3. best system (Omitting lots of items here.....)
7. of government

We can then add the words that follow the word pairs into an array, where the probabilities will be set by the number of occurances of the following words

`
{
    "Democracy is": ["the"],
    "is the" : ["best", "worst"], #<-- Here is the magic, we have multiple options. 
    "the best" : ["system"],
}
`

If we are able to build the hash up from the corpus we can apply the rules for our sentences to generate new sentences. The four valid outputs from this are: 

1. Democracy is the **best** system of government. 
2. Democracy is the **worst** system of government.
3. Dictatorship is the **best** system of government.
4. Dictatorship is the **worst** system of government. 

This is a very small example, but demonstrates the main concepts pretty well. 

## Building a model
To solve the problem of text generation, we'll break the problem up into two steps: 
1. Building the model from the corpus
2. Generating new text from the model

## Activity 3 (10 minutes)
For this activity, we're going to code a simple function that builds our model. The function below takes a corpus of text as an input parameter and builds the hash of data for the model. There are some errors which need fixing before it works successfully

In [4]:
import pprint
def simple_model(corpus):
    model = {} #An empty hash to store the values as they are added
    
    words = corpus.split() #Split the corpus up into individual words
    
    for i in range(len(words)-2): 
        phrase = "{} {}".format(words[i], words[i+1])
        next_word = words[i+2]
        
        if phrase not in model:
            model[phrase] = []
        
        model[phrase].append(next_word)
        
    return model

Lets test the code is working on our simple example.

In [38]:
sample_corpus = "Democracy is the best system of government. Dictatorship is the worst system of government."
model = simple_model(sample_corpus)
pprint.pprint(model)

#Add in some test inputs of your own here. 
test_input = ""
model = simple_model(test_input)
pprint.pprint(model)

{'Democracy is': ['the'],
 'Dictatorship is': ['the'],
 'best system': ['of'],
 'government. Dictatorship': ['is'],
 'is the': ['best', 'worst'],
 'of government.': ['Dictatorship'],
 'system of': ['government.', 'government.'],
 'the best': ['system'],
 'the worst': ['system'],
 'worst system': ['of']}
{}


## Activity 3a (optional)
If you have time, complete the function below which allows the number of states to be set. 

In [11]:
def tuneable_model(corpus, states=2):
    model = {} #An empty hash to store the values as they are added
    
    words = corpus.split() #Split the corpus up into individual words
    
    for i in range(len(words)-states-1): 
        phrase = " ".join(words[i:i+states])
        next_word = words[i+states]
        
        if phrase not in model:
            model[phrase] = []
        
        model[phrase].append(next_word)
        
    return model

And again lets test the model in two ways:

In [ ]:
two_state_model = tuneable_model(sample_corpus,2)
print("Two state model")
print("================")
pprint.pprint(two_state_model)

print()

three_state_model = tuneable_model(sample_corpus,3)
print("Three state model")
print("================")
pprint.pprint(three_state_model)

## Generating text from the model
Now that you have a model in place you are ready to write a function that can process the model and generate new text from a starting point. Remember the two simple rules that we put in place around sentences: 

1. Sentences start with a capital letter
2. Sentences end with a full stop. 

Were going to need to expand on them slightly:

1. Sentences start with a capital letter **or an @ symbol** (To allow us to work with twitter data) 
2. Sentences end with a full stop **or an exclamation mark or question mark ** (Again twitter is a fiery place) 

## Activity 4 (10 minutes) 

Its your turn to implement some code that works on this now. In order to make this a little easier, I've broken it down into a number of functions that we can combine into a single `generate_text` model at the end.

In [15]:
#This function reads the str parameter and decides if it is a valid starting point (i.e starts with a capital letter)
def is_valid_start(str):
    return str[0].isupper() or str[0] == "@"

#This function reads a string parameter and decides if it is a valid end point
def is_valid_end(str):
    return str[-1] == "." or str[-1] == "!" or str[-1] == "?"

#This function should produce the number of words in the string provided
def words_in_string(str):
    return len(str.split())

def generate_text(model, sentences_to_generate=4, minimum_sentence_length=4):
    
    text = [] #An array of sentences
    words_added = 0
    for i in range(sentences_to_generate):
        sentence = ""
        #Choose a sentence starting location from the keys of the hash. Remember the rules!
        starting_point = random.choice(list(model.keys()))
        
        while is_valid_start(starting_point) == False:
            starting_point = random.choice(list(model.keys()))
        
        sentence += starting_point
        
        #Loop infinitely until we have reached the end of a word 
        while True: 
            key = " ".join(sentence.split()[-2:])
            
            next_generated_word = random.choice(model[key])
            
            sentence += (" " + next_generated_word)
            
            if words_in_string(sentence) >= minimum_sentence_length and is_valid_end(sentence):
                text.append(sentence)
                break
        
        
    return '\n'.join(text)

Now test the generation, using the original two sentence sample corpus. 

In [69]:
model = simple_model(sample_corpus)
new_sentences = generate_text(model, 3)

print(new_sentences)


Democracy is the worst system of government.
Dictatorship is the best system of government.
Dictatorship is the best system of government.


Hopefully, you now have generated some sentences that were not in the original corpus. Given the limited size of the inputs there really is not much that the system can do. Lets move on to experimenting with the system as a whole. 




## The system in action.......

You've worked really hard to build the system above. A standard part of any machine learning project is the collection and cleansing of a dataset to train on. This can be a time consuming process so we have put together some pre-prepared data for this session. We have developed the following datasets:
1. Taylor Swift lyrics
2. Donald Trump Tweets
3. A collection of scientific wikipedia articles
4. Some BBC articles on economics

We have wrapped these up into a single function called `fetch_corpus` that you can use to access the data sets as:
1. `fetch_corpus('taylor')`
2. `fetch_corpus('trump')`
3. `fetch_corpus('wikipedia')`
4. `fetch_corpus('bbc')`

These all return a string that you can feed into your models. Right then time to experiment! 

## Experimentation
If your code has worked so far with all of the tests, then you are good to go! If you've gone off-piste and haven't quite got it working then you can uncomment the import line in the code below to bring in some pre-written versions of the functions for experimentation. 

In [16]:
#Uncomment the line(s) below to include the prewritten functions that you need
#from markov_helper import simple_model
#from markov_helper import tuneable_model

from markov_helper import fetch_corpus

corpus = fetch_corpus('taylor') #Choose a corpus 

#Setup the model 
model = simple_model(corpus)

#Generate text
new_sentences = generate_text(model, 10)

print(new_sentences)

I'm ever gonna feel that way again.But I don't know if I should.I knew his world moved too fast and burned too bright.But I just thought, how can the devil be pulling you toward someone who looks so much like an angel when he saw me.I guess you didn't care, and I guess I liked that.And when I fell hard you took a step back.Without me, without me, without me, without me, without me.And he's long gone when he's next to me.And I realize the joke is on me, yeah!I knew you were trouble when you walked in.So shame on me now.Flew me to places I'd never been.Til you put me down, oh!I knew you were trouble when you walked in.So shame on me now.Flew me to places I'd never known.Missing him was blue like I'd never been.Now I'm lying on the street.A new notch in your belt is all I'll ever be.And now I see.He was long gone when he's next to me.And I realize the blame is on me, yeah!I knew you were never a saint I've loved in shades of wrong.We learn to live with the flow.But you're friction.This sl

## Conclusions